In [3]:
#テキストの属性をもとに段落を判別しつつテキストデータを取得
#ヘッダ・フッタ等をテキストブロックの属性情報（特に位置情報）で除外
#テキストを取得する範囲も、テキストブロックの開始位置の座標情報をもとに、強制的に読み込み順序を制御
#（リスト形式で、x軸の閾値を設定：ブランクのリストであればブロックの順番通りに読み込み）
#各ブロックの座標の分布は、Extract_Pdf_Attrib.ipynbで確認可能

import fitz  # PyMuPDF
import re

def clean_text_by_attributes(page, header_y=None, footer_y=None, left_margin_x=None, right_margin_x=None, x0_ranges=None):
    text = ""
    blocks = page.get_text("dict")["blocks"]

    def get_range_index(x0, ranges):
        """x0値がどの範囲に属するかを判定"""
        for i, bound in enumerate(ranges):
            if x0 < bound:
                return i
        return len(ranges)  # 範囲外は最後の区分

    # 各ブロックにLine1のbbox_x0を取得して分類
    sorted_blocks = {i: [] for i in range(len(x0_ranges) + 1)}  # 範囲ごとのリスト
    for block_index, block in enumerate(blocks):
        if "lines" not in block or not block["lines"]:
            continue
        # Line1のbbox_x0を取得
        line1_bbox_x0 = block["lines"][0]["spans"][0]["bbox"][0] if block["lines"][0]["spans"] else None
        if line1_bbox_x0 is not None:
            range_index = get_range_index(line1_bbox_x0, x0_ranges)
            sorted_blocks[range_index].append((block_index, block))  # block_indexを保持

    # 各範囲ごとのブロックを番号順に処理
    previous_font = None
    previous_size = None
    previous_color = None
    previous_bbox = None
    previous_text_type = None

    for range_index in sorted_blocks:
        for block_index, block in sorted_blocks[range_index]:
            # 各ブロックを処理
            for line in block["lines"]:
                for span in line["spans"]:
                    bbox = span["bbox"]  # [x0, y0, x1, y1]
                    content = span["text"]
                    font = span["font"]
                    size = span["size"]
                    color = span["color"]
                    text_type = span.get("text_type", "text")  # デフォルトで"text"

                    # テキスト位置によるフィルタリング
                    if bbox[3] is not None and header_y is not None and bbox[3] < header_y:  # ヘッダの除外
                        continue
                    if bbox[1] is not None and footer_y is not None and bbox[1] > footer_y:  # フッタの除外
                        continue
                    if bbox[2] is not None and left_margin_x is not None and bbox[2] < left_margin_x:  # 左マージンの除外
                        continue
                    if bbox[0] is not None and right_margin_x is not None and bbox[0] > right_margin_x:  # 右マージンの除外
                        continue

                    # 段落区切りの判定
                    threshold = size * 2.001  # フォントサイズに基づく閾値
                    if (
                        (previous_bbox and abs(bbox[3] - previous_bbox[3]) > threshold) or  # 縦位置の大きな変化
                        previous_text_type != text_type
                    ):
                        text += "\n\n"  # 新しい段落の開始
                    else:
                        text += ""  # 同じ段落と見なして連結

                    # \nの直前にある半角スペースを除外
                    content = re.sub(r" \n", "\n", content)
                    content = re.sub(r"^ ", "", content)

                    # テキストを追加
                    text += content

                    # 状態を更新
                    previous_font = font
                    previous_size = size
                    previous_color = color
                    previous_bbox = bbox
                    previous_text_type = text_type

    # 最終的な改行の調整
    text = re.sub(r"\n{3,}", "\n\n", text).strip()
    return text




# PDFファイルのパス
base_dir = "ir/2025/"
#base_dir = "ir/2024/"
#pdf_dir =  "34070_旭化成"
#pdf_filename = "24jp.pdf"
#pdf_dir =  '41880_三菱ケミカルグループ'
#pdf_filename = "23.pdf"
pdf_dir =  '40040_レゾナック・ホールディングス'
pdf_filename = 'pdf-sustainability-report-integratedreport-RESONAC24J_spread.pdf'

pdf_path = base_dir + pdf_dir + "/" + pdf_filename

# PDFファイルのオープン
doc = fitz.open(pdf_path)

# フィルタリング範囲の指定（適宜調整）
#旭化成
#header_y = 37  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = None  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#三菱ケミカル
#header_y = 24  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
#footer_y = 652  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
#left_margin_x = 174  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
#right_margin_x = 997  # 右マージン除外のbbox_x0閾値（範囲内の最小値）

#レゾナック
header_y = 26  # ヘッダ除外のbbox_y1閾値（ヘッダ範囲内の最大値）
footer_y = 651  # フッタ除外のbbox_y0閾値（フッタ範囲内の最小値）
left_margin_x = None  # 左マージン除外のbbox_x1閾値（範囲内の最大値）
right_margin_x = None  # 右マージン除外のbbox_x0閾値（範囲内の最小値）


# PDFからテキストを抽出し、クリーニング
cleaned_text = ""

# x0 の閾値を設定
#x0_ranges = []  
#レゾナック
x0_ranges =[585, 1070]
#x0_ranges =[281, 585, 828, 1070]


for page_number, page in enumerate(doc, start=0):
    # ページ開始タグ
    cleaned_text += f"<page {page_number}>\n"
    # ページ内容（新しいx0_rangesによる処理）
    cleaned_text += clean_text_by_attributes(page, header_y, footer_y, left_margin_x, right_margin_x, x0_ranges=x0_ranges) + "\n"
    # ページ終了タグ
    cleaned_text += f"</page {page_number}>\n\n"

# 出力ファイルパス
output_file_path = pdf_path[:len(pdf_path)-4] + ".txt"

# ファイルへの書き込み
with open(output_file_path, "w", encoding="utf-8") as f:
    f.write(cleaned_text)

print(f"ファイルが保存されました: {output_file_path}")


ファイルが保存されました: ir/2025/40040_レゾナック・ホールディングス/pdf-sustainability-report-integratedreport-RESONAC24J_spread.txt
